<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L11_1_Temporal_Recommender_Systems_Practiceipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-06-11 03:40:55--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  2.67MB/s    in 0.3s    

2024-06-11 03:40:56 (2.67 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [2]:
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
import torch
users, items, ratings, times = [], [], [], []

with open("ml-latest-small/ratings.csv", "r") as f:
    print(f.readline())

    for line in f:
        uid, iid, rating, timestamp = line.split(",")
        users.append(int(uid))
        items.append(int(iid))
        ratings.append(float(rating))
        times.append(int(timestamp))

n_ratings = len(users)

users = torch.LongTensor(users)
items = torch.LongTensor(items)
ratings = torch.FloatTensor(ratings)
times = torch.LongTensor(times)

userId,movieId,rating,timestamp



 train, test 분리

In [5]:
train_size = int(n_ratings * 0.8)
shuffled = torch.randperm(n_ratings)

users_train = users[shuffled[:train_size]].to(device)
items_train = items[shuffled[:train_size]].to(device)
ratings_train = ratings[shuffled[:train_size]].to(device)
times_train = times[shuffled[:train_size]].to(device)

users_test = users[shuffled[train_size:]].to(device)
items_test = items[shuffled[train_size:]].to(device)
ratings_test = ratings[shuffled[train_size:]].to(device)
times_test = times[shuffled[train_size:]].to(device)

In [6]:
print(users_train.shape)
print(items_train.shape)
print(ratings_train)
print(times_train)

#shape는 다 80668

torch.Size([80668])
torch.Size([80668])
tensor([3.0000, 0.5000, 4.0000,  ..., 4.0000, 4.5000, 5.0000], device='cuda:0')
tensor([1258418710, 1518197698,  953610692,  ..., 1460136755, 1180447438,
         967920117], device='cuda:0')


 Bias 모델 파라미터 생성

In [7]:
n_users = max(users) + 1
n_items = max(items) + 1

mean = ratings_train.mean()
b_u = torch.zeros(n_users, requires_grad=True, device=device)
b_i = torch.zeros(n_items, requires_grad=True, device=device)

lmd = 0.00003
# optim = torch.optim.Adam([b_u, b_i], lr=0.1)
optim = torch.optim.Adam([b_u, b_i], lr=0.1, weight_decay=0.00006)
criteria = torch.nn.MSELoss()

for epoch in range(1000):
    h = mean + b_u[users_train] + b_i[items_train]
    reg = lmd * ((b_u ** 2).sum() + (b_i ** 2).sum())
    cost = criteria(h, ratings_train) + reg

    optim.zero_grad()
    cost.backward()
    optim.step()

    with torch.no_grad():
        h_train = mean + b_u[users_train] + b_i[items_train]
        cost_train = criteria(h_train, ratings_train)

        h_test = mean + b_u[users_test] + b_i[items_test]
        cost_test = criteria(h_test, ratings_test)

        if epoch % 100 == 0:
            print(epoch, cost_train.item(), cost_test.item())

0 0.9509391188621521 0.9687032699584961
100 0.6682974696159363 0.749301552772522
200 0.6683434247970581 0.7492996454238892
300 0.6683433651924133 0.7492996454238892
400 0.668343186378479 0.749298632144928
500 0.6683435440063477 0.7492994666099548
600 0.6683433651924133 0.7492994666099548
700 0.6683434247970581 0.7492995858192444
800 0.6683413982391357 0.7492843270301819
900 0.6683398485183716 0.749280571937561


In [8]:
print(b_u.shape)
print(b_i.shape)
print(b_u[users_train].shape)
print(b_i[users_train].shape)

torch.Size([611])
torch.Size([193610])
torch.Size([80668])
torch.Size([80668])


In [9]:
print(b_u[users_train])
print(h)

tensor([-0.3051,  0.2166,  0.5511,  ...,  0.2166,  0.1510,  0.1029],
       device='cuda:0', grad_fn=<IndexBackward0>)
tensor([2.9340, 3.5226, 3.8239,  ..., 3.4903, 3.9972, 3.8080], device='cuda:0',
       grad_fn=<AddBackward0>)


## Temporal Item Bias
전체 시간을 n_bins(=30)개의 구간(bin)으로 나누고,
각 구간마다 item bias 생성

In [10]:
n_users = max(users) + 1
n_items = max(items) + 1

n_bins = 30 # 몇 구간으로 나눌지
tmin = min(times)
trange = max(times) - tmin + 1

# 전체를 n_bins로 봤을 때 t가 어느 슬롯에 있는지 나타냄.
def timetobin(t):
    return (t - tmin) * n_bins // trange

mean = ratings_train.mean()
b_u = torch.zeros(n_users, requires_grad=True, device=device)
b_i = torch.zeros(n_items, requires_grad=True, device=device)
b_it = torch.zeros(n_items, n_bins, requires_grad=True, device=device)

optim = torch.optim.Adam([b_u, b_i], lr=0.1, weight_decay=0.00006)
optim_it = torch.optim.Adam([b_it], lr=0.1, weight_decay=0.0011)

criteria = torch.nn.MSELoss()

for epoch in range(1000):
    h = mean + b_u[users_train] + b_i[items_train]
    h += b_it[items_train, timetobin(times_train)]
    # reg = lmd * ((b_u ** 2).sum() + (b_i ** 2).sum())
    cost = criteria(h, ratings_train)

    optim.zero_grad()       # b_u, b_i의 기울기 초기화
    optim_it.zero_grad()    # b_it의 기울기 초기화
    cost.backward()         # 기울기 계산
    optim.step()            # b_u, b_i 업데이트
    optim_it.step()         # b_it 업데이트

    with torch.no_grad():
        h_train = mean + b_u[users_train] + b_i[items_train]
        h_train += b_it[items_train, timetobin(times_train)]
        cost_train = criteria(h_train, ratings_train)

        h_test = mean + b_u[users_test] + b_i[items_test]
        h_test += b_it[items_test, timetobin(times_test)]
        cost_test = criteria(h_test, ratings_test)

        if epoch % 100 == 0:
            print(epoch, cost_train.item(), cost_test.item())

0 0.8502684235572815 0.9480273723602295
100 0.6193946003913879 0.7452196478843689
200 0.6194182634353638 0.7452303767204285
300 0.6194182634353638 0.7452306151390076
400 0.6194186806678772 0.7452344298362732
500 0.6194183230400085 0.7452320456504822
600 0.6194183826446533 0.7452340722084045
700 0.6194185018539429 0.7452276945114136
800 0.6194179058074951 0.7452341318130493
900 0.6194093823432922 0.7452341318130493


In [11]:
a = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
print(a)
a[[0,1,2],[2,1,0]]

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


tensor([3, 5, 7])

In [12]:
n_cp = 5
user_cps = torch.zeros(n_users, n_cp)

for uid in range(1, n_users):
    times_u = times_train[users_train == uid]
    tmin_u = min(times_u)
    trange_u = max(times_u) - tmin_u
    for i in range(n_cp):
        user_cps[uid, i] = tmin_u + (i * trange_u) // (n_cp-1)

user_cps = user_cps.to(device)

In [13]:
def ut_score(users, times, b_ut, gamma):
    exps = torch.exp(-gamma * torch.abs(user_cps[users] - times.view(-1, 1)))
    nom = (exps * b_ut[users]).sum(dim=1)
    denom = exps.sum(dim=1)
    return nom / denom

In [14]:

n_users = max(users) + 1
n_items = max(items) + 1
n_bins = 30
n_cp = 5
tmin = min(times)
trange = max(times) - tmin + 1

def ut_score(users, times, b_ut, gamma):
    exps = torch.exp(-gamma * torch.abs(user_cps[users] - times.view(-1, 1)))
    nom = (exps * b_ut[users]).sum(dim=1)
    denom = exps.sum(dim=1)
    return nom / denom

def timetobin(t):
    return (t - tmin) * n_bins // trange

mean = ratings_train.mean()
b_u = torch.zeros(n_users, requires_grad=True, device=device)
b_i = torch.zeros(n_items, requires_grad=True, device=device)
b_it = torch.zeros(n_items, n_bins, requires_grad=True, device=device)
b_ut = torch.zeros(n_users, n_cp, requires_grad=True, device=device)

optim = torch.optim.Adam([b_u, b_i], lr=0.1, weight_decay=0.00006)
optim_it = torch.optim.Adam([b_it], lr=0.1, weight_decay=0.0011)
optim_ut = torch.optim.Adam([b_ut], lr=0.1, weight_decay=0.0)

criteria = torch.nn.MSELoss()
gamma = 9e-9

for epoch in range(1000):
    h = mean + b_u[users_train] + b_i[items_train]
    h += b_it[items_train, timetobin(times_train)]
    h += ut_score(users_train, times_train, b_ut, gamma)
    # reg = lmd * ((b_u ** 2).sum() + (b_i ** 2).sum())
    cost = criteria(h, ratings_train)

    optim.zero_grad()       #b_u, b_i의 기울기 초기화
    optim_it.zero_grad()    #b_it의 기울기 초기화
    optim_ut.zero_grad()    #b_ut의 기울기 초기화
    cost.backward()         #기울기 계산
    optim.step()            # b_u, b_i 업데이트
    optim_it.step()         # b_it 업데이트
    optim_ut.step()         # b_ut 업데이트

    with torch.no_grad():
        h_train = mean + b_u[users_train] + b_i[items_train]
        h_train += b_it[items_train, timetobin(times_train)]
        h_train += ut_score(users_train, times_train, b_ut, gamma)
        cost_train = criteria(h_train, ratings_train)

        h_test = mean + b_u[users_test] + b_i[items_test]
        h_test += b_it[items_test, timetobin(times_test)]
        h_test += ut_score(users_test, times_test, b_ut, gamma)
        cost_test = criteria(h_test, ratings_test)

        if epoch % 100 == 0:
            print(epoch, cost_train.item(), cost_test.item())

0 0.8148093223571777 0.9123657941818237
100 0.6112995743751526 0.7360290288925171
200 0.6098133325576782 0.7349340319633484
300 0.6090230345726013 0.7344127297401428
400 0.6085614562034607 0.7342014312744141
500 0.6082574725151062 0.7340958714485168
600 0.6080327033996582 0.7340258359909058
700 0.6078512072563171 0.7339572310447693
800 0.6076995134353638 0.7339202165603638
900 0.6075661778450012 0.7338715195655823
